<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 119MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [4]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [6]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [7]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [8]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.5)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [9]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [10]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [11]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [13]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [14]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
      
        self.data = data
        self.size = len(data[feature_column])

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens 

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec and len(w) > 3] # Получи эмбеддинги слов и усредни их

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [15]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [16]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

800


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [17]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"]) # Обучи PCA на эмбеддингах слов

In [18]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [19]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [21]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [22]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score as acc_score


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        optimizer.zero_grad()
        # Получи предсказания модели
        y_pred = model(features)
        loss = criterion(y_pred, targets)
        loss.backward()
        optimizer.step()
        # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            y_pred = model(features)
            loss = criterion(y_pred, targets)

            predicted = torch.argmax(y_pred, 1).detach().cpu().numpy()
            acc = acc_score(predicted, targets.detach().cpu().numpy()) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


Сначала попробуем использовать градиентный бустинг на базовых параметрах и посмотрим на качество, которое он дает.

Подготовим данные для обучения 

In [ ]:
X_train = np.zeros(shape=(1, 300))
y_train = np.array([])

for batch in tqdm(train_loader):
    X_train = np.append(X_train, batch["features"].numpy(), axis=0)
    y_train = np.append(y_train, batch["targets"].numpy())

X_train = X_train[1:]

In [ ]:
X_train.shape, y_train.shape

((640000, 300), (640000,))

In [ ]:
X_test = np.zeros(shape=(1, 300))
y_test = np.array([])

for batch in tqdm(valid_loader):
    X_test = np.append(X_test, batch["features"].numpy(), axis=0)
    y_test = np.append(y_test, batch["targets"].numpy())

X_test = X_test[1:]

In [ ]:
X_test.shape, y_test.shape

((160000, 300), (160000,))

Установим и обучим базовую модель градиентноо бустинга

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier as CBC

classifier = CBC(task_type='GPU')
classifier.fit(X_train, y_train)

Оценим качество 

In [ ]:
classifier.score(X_test, y_test)

0.72623125

Теперь обучим простую линейную модель

In [24]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = nn.Sequential(
        nn.Linear(vector_size, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
        nn.Linear(128, 32),
        nn.BatchNorm1d(32),
        nn.LeakyReLU(),
        nn.Linear(32, num_classes),
        nn.Sigmoid())

In [25]:
# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [26]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5513048464325583, 'Test Acc': 0.7447937400477707}



{'Test Loss': 0.5459825700255716, 'Test Acc': 0.7503172273089171}


In [27]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5455085491890188, 'Test Acc': 0.7501561001438619}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [28]:
TOK = nltk.WordPunctTokenizer()

In [29]:
def preprocess(text):
    line = TOK.tokenize(text.lower())
    filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
    return ' '.join(filtered_line)

In [30]:
df = data
corpus = df['text'].apply(preprocess)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf

tfidf = Tfidf()
matrix = tfidf.fit_transform(corpus)
words = tfidf.get_feature_names()

In [32]:
S1 = set(words)
S2 = set(corpus[231].split())
idxs = matrix[1].nonzero()[1]
intersec = S1.intersection(S2)

In [34]:
D = {token: idx for token, idx in zip(intersec, idxs)}
k = []
for w in corpus[231].split():
    k.append(D[w])
print(k)

[556545, 206729, 98015, 498832, 570376, 57462, 477023, 380729]


In [35]:
from torch.utils.data import Dataset, random_split


class TwitterDataset_TFIDF(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, matrix, words):
        self.tokenizer = nltk.WordPunctTokenizer()
      
        self.data = data
        self.size = len(data[feature_column])

        self.matrix = matrix
        self.corpus = set(words)

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0) + np.mean(matrix) 
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens, item)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens 

    def get_embeddings_(self, tokens, item):
        # Получим эмбеддинги слов с помощью word2vec и добавим к ним tf-idf эмбеддинги 
        
        tokens_set = set(tokens)
        intersec = self.corpus.intersection(tokens_set)
        idxs = matrix[item].nonzero()[1]
        ifidf_encoding = {word: idx for word, idx in zip(intersec, idxs)}
         
        #print('set: ', tokens_set, 'list: ', tokens)

        embeddings = []
        for w in tokens:
            word_vector = np.zeros(self.word2vec.vector_size) 
            if w in ifidf_encoding: word_vector += self.matrix[item, ifidf_encoding[w]]
            if len(w) > 3 and w in word2vec:
               word_vector = self.word2vec.get_vector(w) - self.mean
               word_vector /= self.std

            embeddings.append(word_vector)  

      
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [36]:
dev_tfidf = TwitterDataset_TFIDF(dev_data, "text", "emotion", word2vec, matrix, words)

In [37]:
from torch.utils.data import DataLoader
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

train_size = math.ceil(len(dev_tfidf) * 0.8)
train_tf, valid_tf = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

train_loader_tf = DataLoader(train_tf, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_tf = DataLoader(valid_tf, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [38]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev_tfidf.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = nn.Sequential(
        nn.Linear(vector_size, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
        nn.Linear(128, 32),
        nn.BatchNorm1d(32),
        nn.LeakyReLU(),
        nn.Linear(32, num_classes),
        nn.Sigmoid())

model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [40]:
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader_tf, e, device)
    log = testing(model, criterion, valid_loader_tf, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5388331823288255, 'Test Acc': 0.7572775676751592}



{'Test Loss': 0.5399632374192499, 'Test Acc': 0.7568670382165605}


In [41]:
test_loader = DataLoader(
    TwitterDataset_TFIDF(test_data, "text", "emotion", word2vec, matrix, words), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5430417030363741, 'Test Acc': 0.7532331461796675}


In [42]:
from torch.utils.data import Dataset, random_split


class TwitterDataset_sum(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
      
        self.data = data
        self.size = len(data[feature_column])

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens 

    def get_embeddings_(self, tokens):
        ##TODO: write embedings preprocessing for nonexistent words

        embeddings = []
        for w in tokens:
            word_vector = np.zeros(self.word2vec.vector_size) 
            if len(w) > 3 and w in word2vec:
               word_vector = self.word2vec.get_vector(w) - self.mean
               word_vector /= self.std
            embeddings.append(word_vector) 

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            context = np.mean(embeddings, axis=0)
            for i in range(len(embeddings)):
                if all(embeddings[i] == 0):
                   embeddings[i] += context
            if len(embeddings.shape) == 1:
               embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [43]:
dev_sum = TwitterDataset_sum(dev_data, "text", "emotion", word2vec)

In [45]:
from torch.utils.data import DataLoader
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

train_size = math.ceil(len(dev_sum) * 0.8)
train_sum, valid_sum = random_split(dev_sum, [train_size, len(dev_sum) - train_size])

train_loader_sum = DataLoader(train_sum, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_sum = DataLoader(valid_sum, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [46]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev_sum.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

model = nn.Sequential(
        nn.Linear(vector_size, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(),
        nn.Linear(128, 32),
        nn.BatchNorm1d(32),
        nn.LeakyReLU(),
        nn.Linear(32, num_classes),
        nn.Sigmoid())

model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [47]:
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader_sum, e, device)
    log = testing(model, criterion, valid_loader_sum, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5508355373030256, 'Test Acc': 0.7448808220541401}



{'Test Loss': 0.5460726166986356, 'Test Acc': 0.7494215266719745}


In [49]:
test_loader = DataLoader(
    TwitterDataset_sum(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5450308526415959, 'Test Acc': 0.7509853041080563}


Сделаем выводы на основании проделанной работы. Модель с If-Idf дает лучший рещультат при обучении нашей модели. Результат потенциально можно улучшить, если взять более глубокую нейросеть и обучать ее дольше, чем две эпохи, однако и в нашем случае результат заметен. Это можно объяснить тем, что при использовании If-Idf модель получает принципиально новую информацию особенно в случае отсутствия слова в word2vec. Таким образов становится возможно извлечь больше из имеющихся данных.

В случае суммирования и поиска (поиска среднего) мы не добавляем новые данные, а тольно приеобразуем уже существующие. Новую информацию модель не получает, следовательно, качество при использовании среднего сопоставимо с качеством при обучении просто на word2vec